# Paper Tutorial

This tutorial uses bayesian optimization techniques to find the minimum of a function $ f ( x )$ on some bounded set $ X $ , on a subset of $ { R } ^ { D }$. The use of Bayesian optimization allows one to construct a probabilistic model for $ f ( x ) $ and then use this model to identify where in $ X $ to next evaluate the function. 

Typical use cases include when trying to find the minimum of a function, when evaluations of $ f ( x )$ are expensive to perform (for example, training machine learning models). This is done by optimizing where to evaluate next, so that the evaluation may be completed with relatively few evaluations.

# Gaussian Processes 

The user would like to find the minimum of difficult non-convex function. First, a series of Gaussian Processes (GPs) are drawn to put a probability distribution over the function. The prior distribution is modeled as a sample from a Gaussian process(GP). 

#### Why use Gaussian Processes?
The GP finds a distribution over the possible functions $ f(x) $. A GP defines a prior over functions, which can be converted into a posterior over functions given data. GPs are frequently used because of their ability to toggle the smoothness as a hyper-parameter through the kernal function applied to the observed $ X $ values, or the covariance matrix. The covariance matrix to ensures that values that are close together in input space will produce output values that are close together. In other words, the GP will consider the smoothness before suggesting points too far away from the other. 

#### What hyperparameters should be used? 
While it is usually an advantage of GPs to be able to toggle the smoothness, this would require chosing a kernal and hyperparameters. The default kernal is the automatic relevance determination (ARD) squared exponential kernel, shown below: 
$$K _ { S E } ( \mathbf { x } ,\mathbf { x ^ { \prime } } ) = \theta _ { 0} \exp \left\{ - \frac { 1} { 2} r ^ { 2} ( \mathbf { x } ,\mathbf { x ^ { \prime } } ) \right\} $$

$$r ^ { 2} \left( \mathbf { x } ,\mathbf { x } ^ { \prime } \right) = \sum _ { d = 1} ^ { D } \left( x _ { d } - x _ { d } ^ { \prime } \right) ^ { 2} / \theta _ { d } ^ { 2}$$

However, this is unrealistically smooth for practical optimization problems. Snoek et al. instead propose the use of the ARD Matern 5/2 kernel:

$$K _ { M 52} \left( \mathbf { x } ,\mathbf { x } ^ { \prime } \right) = \theta _ { 0} \left( 1+ \sqrt { 5r ^ { 2} \left( \mathbf { x } ,\mathbf { x } ^ { \prime } \right) } + \frac { 5} { 3} r ^ { 2} \left( \mathbf { x } ,\mathbf { x } ^ { \prime } \right) \right) \exp \left\{ - \sqrt { 5r ^ { 2} \left( \mathbf { x } ,\mathbf { x } ^ { \prime } \right) } \right\}$$

####  paul this makes no sense to me:  (about the eqn above). Can I cut?
"This covariance function results in sample functions which are twice-differentiable, an assumption that corresponds to those made by, e.g., quasi-Newton methods, but without requiring the smoothness of the squared exponential"

http://ml.dcs.shef.ac.uk/gpss/gpws14/KernelDesign.pdf


# Acquisition Function

Once sampling functions with GPs, this prior and these data induce a posterior over functions. The GP has proposed several different functions. Now, where these functions are proposed dictates the the acquisition function. The acquisition function ($a : \mathcal { X } \rightarrow \mathbb { R } ^ { + }$) determines what point in X should be evaluated next via a proxy optimization ($\mathbf { X } _ { n e x t } = \arg \max _ { x } a ( x )$), given by the GPs proposals. 

Though three different acquisition functions are proposed, this tutorial focuses on the Expected Improvement (EI) criterion: 
 
$$ a _ { E l } \left( \mathbf { x } ; \left\{ \mathbf { x } _ { n } ,y _ { n } \right\} ,\theta \right) = \sigma \left( \mathbf { x } ; \left\{ \mathbf { x } _ { n } ,y _ { n } \right\} ,\theta \right) ( \gamma ( \mathbf { x } ) \Phi ( \gamma ( \mathbf { x } ) ) + \mathcal { N } ( \gamma ( \mathbf { x } ) ; 0,1) )$$

Snoek et al. chose to maximize the expected improvement (EI) over the current best when chosing the next location to evaluate. 

# ( Paul - little confused here feel free to edit/)
 

# MC 

This method allows the user to parallelize optimization procedures. Snoek et al compute Monte Carlo estimates of the acquisiton function under different possible results from pending function evaluations. This allows the algorithm to identify what $X$ should be evaluated next, even while a set of points are currently being evaluated (parallelized).

This is done as the same acquisition function cannot be used again, or a pending experiment would repeate and thus waste computing resources (cost, time, etc). 

A new point is proposed to be evaluated next based on the expected acquisition function under all possible outcomes of these pending evaluations:

$$\hat { a } \left( \mathbf { x } ; \left\{ \mathbf { x } _ { n } ,y _ { n } \right\} ,\theta ,\left\{ \mathbf { x } _ { j } \right\} \right) =$$ 
$$\int _ { \mathbb { R } ^ { J } } a \left( \mathbf { x } ; \left\{ \mathbf { x } _ { n } ,y _ { n } \right\} ,\theta ,\left\{ \mathbf { x } _ { j } ,y _ { j } \right\} \right) p \left( \left\{ y _ { j } \right\} _ { j = 1} ^ { J } | \left\{ \mathbf { x } _ { j } \right\} _ { j = 1} ^ { J } ,\left\{ \mathbf { x } _ { n } ,y _ { n } \right\} _ { n = 1} ^ { N } \right) \text{d} y _ { 1} \cdots \text{d} y _ { J }$$

(I ignored modeling costs- shall I put it back in?)

We would like to be able to decide what x should be evaluated next, even while a set of points are being evaluated.

We propose a sequential strategy that takes advantage of the tractable inference properties of the Gaussian process to compute Monte Carlo estimates of the acquisiton function under different possible results from pending function evaluations.


## should I cut this eqn

$$\hat { a } \left( \mathbf { x } ; \left\{ \mathbf { x } _ { n } ,y _ { n } \right\} ,\theta ,\left\{ \mathbf { x } _ { j } \right\} \right) =$$
$$ \int _ { \mathbb { R } ^ { J } } a \left( \mathbf { x } ; \left\{ \mathbf { x } _ { n } ,y _ { n } \right\} ,\theta ,\left\{ \mathbf { x } _ { j } ,y _ { j } \right\} \right) p \left( \left\{ y _ { j } \right\} _ { j = 1} ^ { J } | \left\{ \mathbf { x } _ { j } \right\} _ { j = 1} ^ { J } ,\left\{ \mathbf { x } _ { n } ,y _ { n } \right\} _ { n = 1} ^ { N } \right) \text{d} y _ { 1} \cdots \text{d} y _ { J } $$

In [ ]:
# Paper Tutorial

This tutorial uses bayesian optimization techniques to find the minimum of a function $ f ( x )$ on some bounded set $ X $ , on a subset of $ { R } ^ { D }$. The use of Bayesian optimization allows one to construct a probabilistic model for $ f ( x ) $ and then use this model to identify where in $ X $ to next evaluate the function. 

Typical use cases include when trying to find the minimum of a function, when evaluations of $ f ( x )$ are expensive to perform (for example, training machine learning models). This is done by optimizing where to evaluate next. 

"This results in a procedure that can find the minimum of difficult non-convex functions with relatively few evaluations, at the cost of performing more computation to determine the next point to try. When evaluations of f(x) are expensive to perform — as is the case when it requires training a machine learning algorithm — then it is easy to justify some extra computation to make better decision"


The tutorial provides "automatic approaches that can optimize the performance of any given learning algorithm to the problem at hand. In this work, we consider this problem through the framework of Bayesian optimization, in which a learning algorithm’s generalization performance is modeled as a sample from a Gaussian process (GP). We show that certain choices for the nature of the GP, such as the type of kernel and the treatment of its hyperparameters, can play a crucial role in obtaining a good optimizer that can achieve expertlevel performance. We describe new algorithms that take into account the variable cost (duration) of learning algorithm experiments and that can leverage the presence of multiple cores for parallel experimentation. We show that these proposed algorithms improve on previous automatic procedures and can reach or surpass human expert-level optimization for many algorithms including latent Dirichlet allocation, structured SVMs and convolutional neural networks."



In [2]:
sources: http://katbailey.github.io/post/gaussian-processes-for-dummies/

SyntaxError: invalid syntax (<ipython-input-2-770b1d8a65f3>, line 1)

In [ ]:
The tutorial provides "automatic approaches that can optimize the performance of any given learning algorithm to the problem at hand. In this work, we consider this problem through the framework of Bayesian optimization, in which a learning algorithm’s generalization performance is modeled as a sample from a Gaussian process (GP). We show that certain choices for the nature of the GP, such as the type of kernel and the treatment of its hyperparameters, can play a crucial role in obtaining a good optimizer that can achieve expertlevel performance. We describe new algorithms that take into account the variable cost (duration) of learning algorithm experiments and that can leverage the presence of multiple cores for parallel experimentation. We show that these proposed algorithms improve on previous automatic procedures and can reach or surpass human expert-level optimization for many algorithms including latent Dirichlet allocation, structured SVMs and convolutional neural networks."

While it is usually an advantage of GPs to be able to toggle the smoothness, a typical kernal is the automatic relevance determination (ARD) squared exponential kernel. This is unrealistically smooth for practical optimization problems. 
Snoek instead proposes the use of the ARD Matern ´ 5/2 kernel:

$$K _ { M 52} \left( \mathbf { x } ,\mathbf { x } ^ { \prime } \right) = \theta _ { 0} \left( 1+ \sqrt { 5r ^ { 2} \left( \mathbf { x } ,\mathbf { x } ^ { \prime } \right) } + \frac { 5} { 3} r ^ { 2} \left( \mathbf { x } ,\mathbf { x } ^ { \prime } \right) \right) \exp \left\{ - \sqrt { 5r ^ { 2} \left( \mathbf { x } ,\mathbf { x } ^ { \prime } \right) } \right\}$$